In [ ]:
# Google Colabでライブラリをアップロードする
from google.colab import files
uploaded = files.upload()

In [ ]:
# Google Colabでドライブのデータを使う
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Google Colabでライブラリをインストールする
!pip install japanize_matplotlib bottleneck tslearn

In [1]:
# 自作関数
import MasterResearchFunction as mr

# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal
import pickle

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from scipy.interpolate import interp1d
from tslearn.metrics import dtw_path
from fastdtw import fastdtw

In [ ]:
#Apple Watchのモーションデータの読み込み
a_yuuma_motion_data = mr.process_apple_watch_csv("datasets/yuuma/20240604/yuuma_motion.csv")
b_sakamoto_motion_data = mr.process_apple_watch_csv("datasets/sakamoto/20240604/sakamoto_motion.csv")
c_watabe_motion_data = mr.process_apple_watch_csv("datasets/watabe/20240605/watabe_motion.csv")
d_nakazawa_motion_data = mr.process_apple_watch_csv("datasets/nakazawa/20240606/nakazawa_motion.csv")
e_okede_motion_data = mr.process_apple_watch_csv("datasets/okeda/20240607/okeda_motion.csv")
# a_yuuma_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_motion.csv")
# b_sakamoto_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_motion.csv")
# c_watabe_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_motion.csv")
# d_nakazawa_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_motion.csv")
# e_okeda_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_motion.csv")

In [2]:
#Tobiiのモーションデータの読み込み
a_yuuma_eye_data = mr.process_tobii_csv("datasets/yuuma/20240604/yuuma_eye.csv")
b_sakamoto_eye_data = mr.process_tobii_csv("datasets/sakamoto/20240604/sakamoto_eye.csv")
c_watabe_eye_data = mr.process_tobii_csv("datasets/watabe/20240605/watabe_eye.csv")
d_nakazawa_eye_data = mr.process_tobii_csv("datasets/nakazawa/20240606/nakazawa_eye.csv")
e_okeda_eye_data = mr.process_tobii_csv("datasets/okeda/20240607/okeda_eye.csv")
# a_yuuma_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_eye.csv")
# b_sakamoto_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_eye.csv")
# c_watabe_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_eye.csv")
# d_nakazawa_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_eye.csv")
# e_okeda_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_eye.csv")

/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:79: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:79: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:79: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:79: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)


In [ ]:
#a_yuumaくんの教師データ読み込み
a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/check")
a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/circle")
a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/cross(new)")
a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/tri")
# a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/check")
# a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/circle")
# a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/cross(new)")
# a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/tri")

In [ ]:
#b_sakamotoくんの教師データ読み込み
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/tri")
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/tri")

In [ ]:
#b_sakamotoくんの教師データ読み込み
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/tri")
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/tri")

In [ ]:
#c_watabeくんの教師データ読み込み
c_watabe_check = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/check")
c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/circle")
c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/cross")
c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/tri")
# c_watabe_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/check")
# c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/circle")
# c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/cross")
# c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/tri")

In [ ]:
#d_nakazawaくんの教師データ読み込み
d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/check")
d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/circle")
d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/cross")
d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/tri")
# d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/check")
# d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/circle")
# d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/cross")
# d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/tri")

In [ ]:
#e_okedaくんの教師データ読み込み
e_okeda_check = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/check")
e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/circle")
e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/cross")
e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/tri")
# e_okeda_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/check")
# e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/circle")
# e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/cross")
# e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/tri")

In [3]:
motion_data = mr.process_apple_watch_csv('/Users/hinase/Downloads/MotionData_20240805_164514.csv')
train_data = mr.process_all_apple_watch_csv_in_directory('/Users/hinase/Downloads/test_circle')

In [4]:
hinase_circle_seg = mr.three_axis_spring(motion_data, train_data, [5, 5, 5], 'acc')

****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****


In [16]:
final = mr.filter_and_combine_segments(hinase_circle_seg, Hz, 1, 5)

In [ ]:
#  pickleファイルを読み込む
with open('/Users/hinase/Downloads/Th-s/d_nakazawa_acc_check_segments4.7new.pkl', 'rb') as f:
    data = pickle.load(f)

segx = data['d_nakazawa_check_segx']
segy = data['d_nakazawa_check_segy']
segz = data['d_nakazawa_check_segz']